Import Data

In [10]:
#import DataFetcher class from file heart_data.py
import DataFetcher #importing the file DataFetcher.py from the folder data
data_fetcher = DataFetcher.DataFetcher("C:\\Users\\kruth\\OneDrive\\Desktop\\Cardiac_Events_ML\\preprocessing\\final_dataset.csv")
X_train = data_fetcher.get_X_train()
X_val = data_fetcher.get_X_val()
X_test = data_fetcher.get_X_test()
y_train = data_fetcher.get_y_train()
y_val = data_fetcher.get_y_val()
y_test = data_fetcher.get_y_test()
features = data_fetcher.get_features()
target_names = data_fetcher.get_target_names()
X = data_fetcher.get_X()
y = data_fetcher.get_y()

AttributeError: 'DataFetcher' object has no attribute 'get_X_val'

Feature Scaling

In [ ]:
#scaling the feature dataset:
from sklearn.preprocessing import StandardScaler 
# fit the Scaler
scaler = StandardScaler() #creating an instance of the StandardScaler class.
#Scaling the values such that the mean is 0 and std deviation is 1.
scaler.fit(X_train) #fitting the scaler to the training set.
X_train = scaler.transform(X_train) #transforming the training set. 
X_val = scaler.trandform(X_val) #transforming the validation set.
X_test = scaler.transform(X_test) #transforming the testing set.

Initial Model

In [ ]:
#import the LogisticRegression class from sklearn.linear_model
from sklearn.linear_model import LogisticRegression

# train the model
lr_1 = LogisticRegression()

#fitting the model to the training set.
lr_1.fit(X_train, y_train) 

# make predictions on the test dataset
y_pred_1 = lr_1.predict(X_test)

#---------------------------------Evaluating the model---------------------------------

# Evaluate the model performance
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print("\n----------Initial Logistic Regression Metrics----------\n")
print("Accuracy:", accuracy_score(y_test, y_pred_1))
print("Precision:", precision_score(y_test, y_pred_1))
print("Recall:", recall_score(y_test, y_pred_1))
print("F1 Score:", f1_score(y_test, y_pred_1))
#meaning of accuracy score: 0.9 means 90% of the predictions are correct

#classification_report
print("\n----------Initial Classification Report----------\n")
from sklearn.metrics import classification_report
cr = classification_report(y_test, y_pred_1)
print(cr)

Hyperparameter Tuning using Grid Search

In [ ]:
#perform grid search to find the best parameters for the model
from sklearn.model_selection import GridSearchCV
#creating a dictionary of parameters to be used in the grid search:
param_grid = {'C': [0.0001,0.001, 0.01], 'penalty': ['l1', 'l2'], 'solver': ['liblinear','saga'], 'max_iter': [100],'random_state': [42]}
#creating an instance of the GridSearchCV class:
grid = GridSearchCV(LogisticRegression(), param_grid, cv = 10, scoring='f1')

#fitting the grid search to the training set:
lr_tuned = grid.fit(X_val, y_val)
#finding the best estimator:
print("\nbest estimator: ", lr_tuned.best_estimator_)
#finding the best score:
print("best score:  ", lr_tuned.best_score_)

Testing of Final Model

In [ ]:
#test the new model with hyperparameters tuned by grid search:
#make predictions on the test dataset
y_pred = lr_tuned.predict(X_test)

# Evaluate the model performance
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print("\n----------Logistic Regression Metrics----------\n")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

#classification_report
print("\n----------Classification Report----------\n")
from sklearn.metrics import classification_report
cr = classification_report(y_test, y_pred)
print(cr)


Confusion Matrix

In [ ]:
#get the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

#visualize the confusion matrix
import matplotlib.pyplot as plt
import seaborn as sns
sns.heatmap(cm, annot=True, fmt="d") #fmt="d" means the values in the matrix are integers  
#annot=True means the values in the matrix are displayed
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()

In [ ]:
#cross validation for 10 folds

from sklearn.model_selection import cross_val_score
scores = cross_val_score(lr_tuned, X_train, y_train, cv=10, scoring="f1") #scoring can be accuracy, precision, recall, f1, roc_auc, etc
#get the scores for each fold
for i in range(10):
    print("Fold", i, ":", scores[i])
    
#get the mean of the scores
print("\nMean:", scores.mean())
#get the standard deviation of the scores
print("Standard Deviation:", scores.std()) #the lower the standard deviation, the better the model
#standard deviation is calculated for the scores of each fold